# Strands Agentsへのカスタムツールの追加

## 概要
この例では、Strands Agentsを使用してカスタムツールを作成するさまざまな方法をガイドします。ローカルのSQLiteデータベースに接続してデータタスクを実行するパーソナルアシスタントのユースケースを構築します。ボーナスとして、`BedrockModel`クラスの`thinking`フィールドを使用したClaude Sonnet 3.7の推論機能の使用方法についてもガイドします

## エージェントの詳細
<div style="float: left; margin-right: 20px;">
    
|機能                |説明                                        |
|--------------------|-------------------------------------------|
|使用するネイティブツール   |current_time, calculator                 |
|作成するカスタムツール|create_appointment, list_appointments      |
|エージェント構造     |シングルエージェントアーキテクチャ              |

</div>


## アーキテクチャ

<div style="text-align:left">
    <img src="images/architecture.png" width="85%" />
</div>

## 主な機能
* **シングルエージェントアーキテクチャ**: この例では、組み込みツールとカスタムツールの両方とやり取りする単一のエージェントを作成します
* **組み込みツール**: Strands Agentのツールの使用方法を学びます
* **カスタムツール**: 独自のツールを作成する方法を学びます
* **基盤となるLLMとしてのBedrockモデル**: 基盤となるLLMモデルとして、Amazon BedrockからAnthropic Claude 3.7を使用します

## セットアップと前提条件

### 前提条件
* Python 3.10以上
* AWSアカウント
* Amazon BedrockでAnthropic Claude 3.7が有効化されていること、[ガイド](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html)
* Amazon Bedrock Knowledge Base、Amazon S3バケット、Amazon DynamoDBを作成する権限を持つIAMロール

それでは、Strands Agentに必要なパッケージをインストールしましょう

In [ ]:
# installing pre-requisites
!pip install -r requirements.txt

### 依存パッケージのインポート

それでは、依存パッケージをインポートしましょう

In [ ]:
import json
import sqlite3
import uuid
from datetime import datetime

from strands import Agent, tool
from strands.models import BedrockModel

## カスタムツールの定義
次に、ローカルのSQLiteデータベースとやり取りするカスタムツールを定義しましょう：
* **create_appointment**: 一意のID、日付、場所、タイトル、説明を持つ新しい個人的な予定を作成
* **list_appointment**: 利用可能なすべての予定をリスト
* **update_appointments**: 予定IDに基づいて予定を更新

### エージェントと同じファイルでツールを定義

Strands Agents SDKでツールを定義する方法は複数あります。1つ目は、関数に`@tool`デコレータを追加し、ドキュメントを提供する方法です。この場合、Strands Agentsは関数のドキュメント、型付け、引数を使用してエージェントにツールを提供します。この場合、エージェントと同じファイルでツールを定義することもできます

In [ ]:
@tool
def create_appointment(date: str, location: str, title: str, description: str) -> str:
    """
    Create a new personal appointment in the database.

    Args:
        date (str): Date and time of the appointment (format: YYYY-MM-DD HH:MM).
        location (str): Location of the appointment.
        title (str): Title of the appointment.
        description (str): Description of the appointment.

    Returns:
        str: The ID of the newly created appointment.

    Raises:
        ValueError: If the date format is invalid.
    """
    # Validate date format
    try:
        datetime.strptime(date, "%Y-%m-%d %H:%M")
    except ValueError:
        raise ValueError("Date must be in format 'YYYY-MM-DD HH:MM'")

    # Generate a unique ID
    appointment_id = str(uuid.uuid4())

    conn = sqlite3.connect("appointments.db")
    cursor = conn.cursor()

    # Create the appointments table if it doesn't exist
    cursor.execute(
        """
    CREATE TABLE IF NOT EXISTS appointments (
        id TEXT PRIMARY KEY,
        date TEXT,
        location TEXT,
        title TEXT,
        description TEXT
    )
    """
    )

    cursor.execute(
        "INSERT INTO appointments (id, date, location, title, description) VALUES (?, ?, ?, ?, ?)",
        (appointment_id, date, location, title, description),
    )

    conn.commit()
    conn.close()
    return f"Appointment with id {appointment_id} created"

### モジュールベースアプローチによるツール定義

ツールをスタンドアロンファイルとして定義し、エージェントにインポートすることもできます。この場合、デコレータアプローチを使用することも、TOOL_SPEC辞書を使用して関数を定義することもできます。フォーマットは、ツール使用のための[Amazon Bedrock Converse API](https://docs.aws.amazon.com/bedrock/latest/userguide/tool-use-examples.html)で使用されるものと類似しています。この場合、必須パラメータや成功・エラー実行の戻り値をより柔軟に定義でき、TOOL_SPEC定義が機能します。

#### デコレータアプローチ

スタンドアロンファイルでデコレータを使用してツールを定義する場合、プロセスはエージェントと同じファイルで定義する場合と非常に似ていますが、後でエージェントツールをインポートする必要があります。

In [ ]:
%%writefile list_appointments.py
import json
import sqlite3
import os
from strands import tool

@tool
def list_appointments() -> str:
    """
    List all available appointments from the database.
    
    Returns:
        str: the appointments available 
    """
    # Check if database exists
    if not os.path.exists('appointments.db'):
        return "No appointment available"
    
    conn = sqlite3.connect('appointments.db')
    conn.row_factory = sqlite3.Row  # This enables column access by name
    cursor = conn.cursor()
    
    # Check if the appointments table exists
    try:
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='appointments'")
        if not cursor.fetchone():
            conn.close()
            return "No appointment available"
        
        cursor.execute("SELECT * FROM appointments ORDER BY date")
        rows = cursor.fetchall()
        
        # Convert rows to dictionaries
        appointments = []
        for row in rows:
            appointment = {
                'id': row['id'],
                'date': row['date'],
                'location': row['location'],
                'title': row['title'],
                'description': row['description']
            }
            appointments.append(appointment)
        
        conn.close()
        return json.dumps(appointments)
    
    except sqlite3.Error:
        conn.close()
        return []


#### TOOL_SPECアプローチ

あるいは、ツールを定義する際にTOOL_SPECアプローチを使用することもできます

In [ ]:
%%writefile update_appointment.py
import sqlite3
from datetime import datetime
import os
from strands.types.tools import ToolResult, ToolUse
from typing import Any

TOOL_SPEC = {
    "name": "update_appointment",
    "description": "Update an appointment based on the appointment ID.",
    "inputSchema": {
        "json": {
            "type": "object",
            "properties": {
                "appointment_id": {
                    "type": "string",
                    "description": "The appointment id."
                },
                "date": {
                    "type": "string",
                    "description": "Date and time of the appointment (format: YYYY-MM-DD HH:MM)."
                },
                "location": {
                    "type": "string",
                    "description": "Location of the appointment."
                },
                "title": {
                    "type": "string",
                    "description": "Title of the appointment."
                },
                "description": {
                    "type": "string",
                    "description": "Description of the appointment."
                }
            },
            "required": ["appointment_id"]
        }
    }
}
# Function name must match tool name
def update_appointment(tool: ToolUse, **kwargs: Any) -> ToolResult:
    tool_use_id = tool["toolUseId"]
    appointment_id = tool["input"]["appointment_id"]
    if "date" in tool["input"]:
        date = tool["input"]["date"]
    else:
        date = None
    if "location" in tool["input"]:
        location = tool["input"]["location"]
    else:
        location = None
    if "title" in tool["input"]:
        title = tool["input"]["title"]
    else:
        title = None
    if "description" in tool["input"]:
        description = tool["input"]["description"]
    else:
        description = None
        
    # Check if database exists
    if not os.path.exists('appointments.db'): 
        return {
            "toolUseId": tool_use_id,
            "status": "error",
            "content": [{"text": f"Appointment {appointment_id} does not exist"}]
        } 
    
    # Check if appointment exists
    conn = sqlite3.connect('appointments.db')
    cursor = conn.cursor()
    
    # Check if the appointments table exists
    try:
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='appointments'")
        if not cursor.fetchone():
            conn.close()
            return {
                "toolUseId": tool_use_id,
                "status": "error",
                "content": [{"text": f"Appointments table does not exist"}]
            }
        
        cursor.execute("SELECT * FROM appointments WHERE id = ?", (appointment_id,))
        appointment = cursor.fetchone()
        
        if not appointment:
            conn.close()
            return {
                "toolUseId": tool_use_id,
                "status": "error",
                "content": [{"text": f"Appointment {appointment_id} does not exist"}]
            }
        
        # Validate date format if provided
        if date:
            try:
                datetime.strptime(date, '%Y-%m-%d %H:%M')
            except ValueError:
                conn.close()
                return {
                    "toolUseId": tool_use_id,
                    "status": "error",
                    "content": [{"text": "Date must be in format 'YYYY-MM-DD HH:MM'"}]
                }
        
        # Build update query
        update_fields = []
        params = []
        
        if date:
            update_fields.append("date = ?")
            params.append(date)
        
        if location:
            update_fields.append("location = ?")
            params.append(location)
        
        if title:
            update_fields.append("title = ?")
            params.append(title)
        
        if description:
            update_fields.append("description = ?")
            params.append(description)
        
        # If no fields to update
        if not update_fields:
            conn.close()
            return {
                "toolUseId": tool_use_id,
                "status": "success",
                "content": [{"text": "No need to update your appointment, you are all set!"}]
            }
        
        # Complete the query
        query = f"UPDATE appointments SET {', '.join(update_fields)} WHERE id = ?"
        params.append(appointment_id)
        
        cursor.execute(query, params)
        conn.commit()
        conn.close()
        
        return {
            "toolUseId": tool_use_id,
            "status": "success",
            "content": [{"text": f"Appointment {appointment_id} updated with success"}]
        }
    
    except sqlite3.Error as e:
        conn.close()
        return {
            "toolUseId": tool_use_id,
            "status": "error",
            "content": [{"text": str(e)}]
        }


それでは、`list_appointments`と`update_appointment`をツールとしてインポートしましょう

In [ ]:
import list_appointments
import update_appointment

## エージェントの作成

カスタムツールを作成したので、最初のエージェントを定義しましょう。そのためには、エージェントが何をすべきか、何をすべきでないかを定義するシステムプロンプトを作成する必要があります。次に、エージェントの基盤となるLLMモデルを定義し、組み込みツールとカスタムツールを提供します。

#### エージェントのシステムプロンプトの設定
システムプロンプトでは、エージェントの指示を定義します

In [ ]:
system_prompt = """You are a helpful personal assistant that specializes in managing my appointments and calendar. 
You have access to appointment management tools, a calculator, and can check the current time to help me organize my schedule effectively. 
Always provide the appointment id so that I can update it if required"""

#### エージェントの基盤となるLLMモデルの定義

次に、エージェントの基盤となるモデルを定義しましょう。Strands AgentsはAmazon Bedrockモデルとネイティブに統合されており、モデルの呼び出し方法を設定する機能を提供します。以下では、`BedrockModel`プロバイダーの簡単な初期化を確認できます。オプション設定の一部はコメントアウトされています。設定オプションとデフォルト値の詳細については、[Strands Agents Bedrock Model Providerドキュメント](https://strandsagents.com/0.1.x/user-guide/concepts/model-providers/amazon-bedrock/)を参照してください。この例では、BedrockからAnthropic Claude 3.7 Sonnetモデルを使用します。

In [ ]:
model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    # region_name="us-east-1",
    # boto_client_config=Config(
    #    read_timeout=900,
    #    connect_timeout=900,
    #    retries=dict(max_attempts=3, mode="adaptive"),
    # ),
    # temperature=0.9,
    # max_tokens=2048,
)

#### 組み込みツールのインポート

エージェントを構築する次のステップは、Strands Agentsの組み込みツールをインポートすることです。Strands Agentsは、オプションパッケージ`strands-tools`で一般的に使用される組み込みツールのセットを提供しています。RAG、メモリ、ファイル操作、コード解釈などのツールがこのリポジトリで利用できます。この例では、エージェントに現在時刻の情報を提供する`current_time`ツールと、数学を行う`calculator`ツールを使用します

In [ ]:
from strands_tools import calculator, current_time

#### エージェントの定義

必要な情報がすべて揃ったので、エージェントを定義しましょう

In [ ]:
agent = Agent(
    model=model,
    system_prompt=system_prompt,
    tools=[
        current_time,
        calculator,
        create_appointment,
        list_appointments,
        update_appointment,
    ],
)

## エージェントの呼び出し

それでは、レストランエージェントを挨拶で呼び出し、その結果を分析しましょう

In [ ]:
results = agent("How much is 2+2?")

#### エージェントの結果の分析

素晴らしい！初めてエージェントを呼び出しました！それでは、結果オブジェクトを調べましょう。まず、エージェントのオブジェクト内でやり取りされたメッセージを確認できます

In [ ]:
agent.messages

次に、結果の`metrics`を分析して、最後のクエリに対するエージェントの使用状況を確認できます

In [ ]:
results.metrics

#### フォローアップ質問でエージェントを呼び出す
それでは、明日の予定を作成しましょう

In [ ]:
results = agent(
    "Book 'Agent fun' for tomorrow 3pm in NYC. This meeting will discuss all the fun things that an agent can do"
)

#### 予定の更新

それでは、この予定を更新しましょう

In [ ]:
results = agent("Oh no! My bad, 'Agent fun' is actually happening in DC")

#### エージェントの結果の分析
エージェントのメッセージと結果のメトリクスを再度見てみましょう

In [ ]:
agent.messages

In [ ]:
results.metrics

#### メッセージからのツール使用の確認

メッセージ辞書内のツール使用を詳しく見てみましょう。後ほど、エージェントの動作を観察・評価する方法を紹介しますが、これがその第一歩です

In [ ]:
for m in agent.messages:
    for content in m["content"]:
        if "toolUse" in content:
            print("Tool Use:")
            tool_use = content["toolUse"]
            print("\tToolUseId: ", tool_use["toolUseId"])
            print("\tname: ", tool_use["name"])
            print("\tinput: ", tool_use["input"])
        if "toolResult" in content:
            print("Tool Result:")
            tool_result = m["content"][0]["toolResult"]
            print("\tToolUseId: ", tool_result["toolUseId"])
            print("\tStatus: ", tool_result["status"])
            print("\tContent: ", tool_result["content"])
            print("=======================")

### アクションが正しく実行されたことの検証
それでは、データベースを確認して、操作が正しく実行されたことを確認しましょう。`Agent`クラスには、`agent.tool.<tool_name>(<tool_params>)`を呼び出すことで、エージェントが初期化されたツールを直接呼び出す機能があります。直接ツール呼び出しは、エージェント自身がそのツールを呼び出す必要なく、エージェントにツールからの情報を提供するのに最適です。この直接ツール呼び出しを使用して、現在の予定をリストできます：

In [ ]:
list_appointments_result = agent.tool.list_appointments()
print(json.dumps(list_appointments_result, indent=2))

ツールの実行結果がToolResult形式になっており、`toolUseId`、実行`status`、レスポンスの`content`が含まれていることがわかります。ツールの結果は次のようにより見やすく表示できます：

In [ ]:
list_appointments_result_text_content = list_appointments_result["content"][0]["text"]
print(json.dumps(json.loads(list_appointments_result_text_content), indent=2))

最後に、直接ツール呼び出しを使用してツールを実行すると、エージェントはこれらの実行をメッセージ履歴に記録します。デフォルトではこれが有効ですが、`Agent`クラスの`record_direct_tool_call`ブール値フラグ属性で無効にすることができます。

In [ ]:
current_time_result = agent.tool.current_time()
print("Current Time direct tool call result:")
print(current_time_result)
current_time_direct_tool_messages = agent.messages[-4:]
print("Current Time direct tool call messages:")
print(current_time_direct_tool_messages)

agent.record_direct_tool_call = False # Set the record_direct_tool_call to False
agent.tool.list_appointments()
after_disable_record_messages = agent.messages[-4:]
print("After disabling record direct tool call messages, history should not have changed:")
print(current_time_direct_tool_messages == after_disable_record_messages)

## 拡張: Extended Thinking
 
Extended thinkingは、サポートされているClaudeファミリーモデルに、複雑なタスクに対する強化された推論機能を活用する能力を提供し、最終的な回答を提供する前に透明な段階的思考プロセスを提供します。thinkingを有効にするには、Bedrock ModelProviderを設定する際に以下の設定を含めることができます。詳細は[AWSのExtended Thinkingに関するドキュメント](https://docs.aws.amazon.com/bedrock/latest/userguide/claude-messages-extended-thinking.html)を参照してください。

In [ ]:
thinking_model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    additional_request_fields={
        "thinking": {
            "type": "enabled",
            "budget_tokens": 2048,
        }
    },
)

`thinking_model`を定義した後、新しい`thinking_agent`を作成して呼び出すことができます：

In [ ]:
thinking_system_prompt = """You are a helpful personal assistant that specializes in managing my appointments and calendar. 
You have access to appointment management tools, a calculator, and can check the current time to help me organize my schedule effectively. 
You think through your problem, step by step, to come up with an answer.
Always provide the appointment id so that I can update it if required"""

thinking_agent = Agent(
    model=thinking_model,
    system_prompt=thinking_system_prompt,
    tools=[
        current_time,
        calculator,
        create_appointment,
        list_appointments,
        update_appointment,
    ],
)

thinking_result = thinking_agent("I want to add a new appointment for tomorrow at 2pm")

エージェントのメッセージを出力することで、extended thinking機能をより詳しく分析できます。Extended thinkingは、エージェントからのレスポンス内で`reasoningContent`ブロックとして表現されます。

In [ ]:
thinking_agent.messages

## お疲れ様でした！
次のモジュールでお会いしましょう。:)